In [161]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
import os

In [162]:
client = bigquery.Client(project='big-sales-data-453023')
query = """
    SELECT * FROM `big-sales-data-453023.sales_data.sales`;
"""
data = client.query(query).to_dataframe()

/Users/siddharthgoradia/Documents/Loblaw-Data-Science/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [163]:
print(data.head())
print(data.info())

   order_id       product  quantity_ordered  price_each           order_date  \
0  176588.0  20in Monitor               1.0         1.0  2019-04-02 04:00:00   
1  176659.0  20in Monitor               1.0         1.0  2019-04-29 14:40:00   
2  176673.0  20in Monitor               1.0         1.0  2019-04-10 08:17:00   
3  176690.0  20in Monitor               1.0         1.0  2019-04-04 18:33:00   
4  176703.0  20in Monitor               1.0         1.0  2019-04-17 10:00:00   

                             purchase_address  
0            765 Cherry St, Seattle, WA 98101  
1             804 Church St, Dallas, TX 75001  
2                952 1st St, Boston, MA 02215  
3      873 Sunset St, San Francisco, CA 94016  
4  748 Washington St, New York City, NY 10001  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185686 entries, 0 to 185685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          18

In [164]:
data['order_date'] = pd.to_datetime(data['order_date'])

In [165]:
train_data = data[data['order_date'] < '2019-10-01'].copy()
test_data = data[(data['order_date'] >= '2019-10-01') & (data['order_date'] <= '2019-12-31')].copy()

print("Train data range:", train_data['order_date'].min(), "-", train_data['order_date'].max())
print("Test data range:", test_data['order_date'].min(), "-", test_data['order_date'].max())


Train data range: 2019-01-01 03:07:00 - 2019-09-30 23:59:00
Test data range: 2019-10-01 00:09:00 - 2019-12-31 00:00:00


In [166]:
def create_time_features(df):
    df = df.copy()
    df['day'] = df['order_date'].dt.day
    df['month'] = df['order_date'].dt.month
    df['week'] = df['order_date'].dt.isocalendar().week.astype(int)
    df['day_of_week'] = df['order_date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
    return df

train_data = create_time_features(train_data)
test_data = create_time_features(test_data)

In [167]:
train_data = train_data.sort_values(['product', 'order_date']).copy()
train_data['lag_1'] = train_data.groupby('product')['quantity_ordered'].shift(1)
train_data['rolling_mean_7'] = train_data.groupby('product')['quantity_ordered']\
    .transform(lambda x: x.shift(1).rolling(window=7, min_periods=1).mean())

for col in ['lag_1', 'rolling_mean_7']:
    train_data[col] = train_data.groupby('product')[col].transform(lambda x: x.fillna(x.mean()))

def compute_test_features(test_df, train_df, lag_days=1, window=7):
    test_dfs = []
    for pid in test_df['product'].unique():
        train_subset = train_df[train_df['product'] == pid].sort_values('order_date')
        history = train_subset.tail(window)
        test_subset = test_df[test_df['product'] == pid].sort_values('order_date')
        
        combined = pd.concat([history, test_subset], ignore_index=True)
        
        combined['lag_1'] = combined.groupby('product')['quantity_ordered'].shift(lag_days)
        combined['rolling_mean_7'] = combined.groupby('product')['quantity_ordered']\
            .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
        
        for col in ['lag_1', 'rolling_mean_7']:
            combined[col] = combined.groupby('product')[col].transform(lambda x: x.fillna(x.mean()))
        
        test_features = combined.iloc[history.shape[0]:].copy()
        test_dfs.append(test_features)
    return pd.concat(test_dfs, ignore_index=True)

test_data = compute_test_features(test_data, train_data, lag_days=1, window=7)

In [168]:
if 'Sales_Value' not in data.columns:
    data['Sales_Value'] = data['quantity_ordered'] * data['price_each']

product_sales = data.groupby("product")["Sales_Value"].sum().sort_values(ascending=False)
top5_products = product_sales.head(5).index.tolist()
print("Top 5 products by Sales Value:", top5_products)

Top 5 products by Sales Value: ['AAA Batteries (4-pack)', 'AA Batteries (4-pack)', 'USB-C Charging Cable', 'Lightning Charging Cable', 'Wired Headphones']


In [169]:
features = ['day', 'month', 'week', 'day_of_week', 'is_weekend', 'price_each', 'lag_1', 'rolling_mean_7']

results = {}
saved_models = {}

for pid in top5_products:
    print(f"\n--- Processing Product {pid} ---")
    
    train_prod = train_data[train_data['product'] == pid].copy()
    test_prod = test_data[test_data['product'] == pid].copy()
    
    if train_prod.shape[0] < 20 or test_prod.shape[0] < 5:
        print(f"Not enough data for Product {pid}. Skipping.")
        continue
    
    X_train = train_prod[features]
    y_train = train_prod['quantity_ordered']
    X_test = test_prod[features]
    y_test = test_prod['quantity_ordered']
    
    models = {
        'RandomForest': RandomForestRegressor(random_state=42)
    }
    
    product_results = {}
    product_models = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        mae = mean_absolute_error(y_test, preds)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        product_results[name] = {'MAE': mae, 'RMSE': rmse}
        print(f"{name} -> MAE: {mae}, RMSE: {rmse}")
        product_models[name] = model
    
    results[pid] = product_results
    saved_models[pid] = product_models


--- Processing Product AAA Batteries (4-pack) ---
RandomForest -> MAE: 4.169062679700953e-05, RMSE: 0.0017167108178730611

--- Processing Product AA Batteries (4-pack) ---
RandomForest -> MAE: 9.963369963369985e-05, RMSE: 0.005925050810434217

--- Processing Product USB-C Charging Cable ---
RandomForest -> MAE: 0.00023303075464029249, RMSE: 0.008399972903356914

--- Processing Product Lightning Charging Cable ---
RandomForest -> MAE: 2.75292498279416e-06, RMSE: 0.00016591940762894803

--- Processing Product Wired Headphones ---
RandomForest -> MAE: 0.0, RMSE: 0.0


In [170]:
os.makedirs("../models", exist_ok=True)

In [171]:
for pid, product_models in saved_models.items():
    for _, model in product_models.items():
        filename = f"../models/{pid.replace(' ', '_').replace('(', '').replace(')', '')}.pkl"
        with open(filename, "wb") as f:
            pickle.dump(model, f)
        print(f"Saved model for {pid} - {name} to {filename}")

Saved model for AAA Batteries (4-pack) - RandomForest to ../models/AAA_Batteries_4-pack.pkl
Saved model for AA Batteries (4-pack) - RandomForest to ../models/AA_Batteries_4-pack.pkl
Saved model for USB-C Charging Cable - RandomForest to ../models/USB-C_Charging_Cable.pkl
Saved model for Lightning Charging Cable - RandomForest to ../models/Lightning_Charging_Cable.pkl
Saved model for Wired Headphones - RandomForest to ../models/Wired_Headphones.pkl
